In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
import helper_scgen as hscg
print(sys.executable)
import anndata as ad
import time
import scgen
import scvi
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
h.print_main_versions()



In [ ]:
pre = "MH47"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
test_adata = sc.read_h5ad(os.path.join("F:\\monkey_IZI\\analysis",
             "cluster_runs",
             "mh89exih-runVAE","output","MH45_c","anndata_objects",
             "MH45_c_test_adata_hyperparameter.h5ad"))

In [ ]:
base_model_path_MH45_c =os.path.join("F:\\monkey_IZI\\analysis",
             "cluster_runs",
             "mh89exih-runVAE","output","MH45_c","models")

In [ ]:
n_latents = [6,8,10,12]
batch_sizes = [8,16,32,64]

In [ ]:
df_eval = pd.DataFrame(columns = ['tool','n_latent','batch_size','MSE_test'])
X = test_adata.X.toarray()
for n_latent in n_latents:
    for batch_size in batch_sizes:
        load_string = os.path.join(
            base_model_path_MH45_c,'nlat' +str(n_latent) + '_bs' +  str(batch_size),
    'MH45_c_scgen_model_nlat' + str(n_latent) + '_bs' + str(batch_size)  + '.pt'
)
        model = scgen.SCGEN.load(load_string)
        reconstructed_X = model.get_decoded_expression(adata=test_adata)
        mse = np.mean((X - reconstructed_X) ** 2)
        row_to_add = pd.DataFrame(['scgen',n_latent,batch_size,mse],index = ['tool','n_latent','batch_size','MSE_test']).T
        df_eval = df_eval.append(row_to_add) 

In [ ]:
df_eval['rank_MSE_test'] = df_eval['MSE_test'].rank(ascending=True).astype(int)
df_eval.to_csv(os.path.join(base_table_path,pre + '_df_eval_hypserach_scgen.csv'))

In [ ]:
df_eval[df_eval['rank_MSE_test'] == 1]

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()